In [1]:
import cv2
import imutils
import pandas as pd
import numpy as np

In [2]:
class ShapeDetector:
    def __init__(self):
        pass

    def detect(self, c):
        # initialize the shape name and approximate the contour
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)

        # if the shape is a triangle, it will have 3 vertices
        if len(approx) == 3:
            shape = "triangle"

        # if the shape is a square or a rectangle, it will have 4 vertices
        elif len(approx) == 4:
            # compute the bounding box of the contour and use the bounding box to compute aspect ratio
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w/float(h)

            # a square will have an aspect ration that is approximately
            # equal to one, otherwise the shape is a rectangle
            shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"

        # if the shape is a pentagon, it will have 5 vertices
        elif len(approx) == 5:
            shape = "pentagon"

        # otherwise, we assume the shape is a circle
        else:
            shape = "circle"

        return shape

In [3]:
image = cv2.imread('Resources/shapes_and_colors.png')
(h, w, d) = image.shape
print("width={}, height={}, depth={}".format(w, h, d))

width=600, height=561, depth=3


In [4]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]

sd = ShapeDetector()

cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# cv2.RETR_EXTERNAL flag retrieves only the extreme outer contours (ignores nested contours), 
# cv2.CHAIN_APPROX_SIMPLE compresses horizontal, vertical, and diagonal segments, leaving only their end points.
cnts = imutils.grab_contours(cnts)

# loop over the contours
for c in cnts:
    # Compute the moments of the contour, which can be used to calculate the centroid (center point).
    M = cv2.moments(c)

    #If the area (M["m00"]) is zero, it is set to 1 to avoid division by zero.
    if (M["m00"] == 0):
        M["m00"]=1    

    # The function cv2.moments() calculates the moments of a binary image (in this case, the contour c). 
    # The moments are a set of scalar values calculated from the image’s pixels’ intensities, 
    # and they provide information about the shape of the object in the image.

    # The moments include spatial(пространственные) moments (M["m00"], M["m01"], M["m10"], etc.) and central moments. 
    # The spatial moments M["m00"], M["m01"], and M["m10"] are used to calculate the centroid of the shape.

    # The centroid (cX, cY) of the shape is calculated as follows:
    
    cX = int(M["m10"] / M["m00"])
    # This line calculates the x-coordinate of the centroid. 
    # M["m10"] is the sum of the products of each pixel’s x-coordinate and its intensity, 
    # and M["m00"] is the sum of the intensities of all the pixels (essentially, the area of the shape for binary images).
    
    cY = int(M["m01"] / M["m00"])
    # This line calculates the y-coordinate of the centroid. 
    # M["m01"] is the sum of the products of each pixel’s y-coordinate and its intensity.

    # get the shape name
    shape = sd.detect(c)
    
    # This line draws the contour on the image in green color.
    cv2.drawContours(image, [c], -1, (0, 255, 0), 2)
    # This line draws a white circle at the centroid.
    cv2.circle(image, (cX, cY), 7, (255, 255, 255), -1)
    # put a text label near the centroid.
    cv2.putText(image, shape, (cX - 20, cY - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # show the image
    cv2.imshow("Image", image)
    cv2.waitKey()